In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
url = "https://www.wapcar.my/cars"

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html')
# soup

In [4]:
brands = [x.get('href') for x in soup.find("ul", {"class":"brand-filter-block"}).find_all('a')]

In [63]:
# brands

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36',
           'Referer':'https://www.wapcar.my/'}

In [46]:
# test 
# import json
# url = "https://www.wapcar.my/cars/audi"
# r = requests.get(url,headers = headers)
# soup = BeautifulSoup(r.content, "html.parser")
# for i in soup.find_all('script', {'type':'application/ld+json'}):
#     data = json.loads(i.get_text())
#     if data['@type'] == 'ImageObject':
#         print(data['mainEntityOfPage'])
#         print("\n\n")
#     # print(data)
#     # print("\n\n")

In [53]:
# for x in brands:
#     print(x)

In [61]:
def crawl(x):
    url = "https://www.wapcar.my" + x
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "html.parser")
    for i in soup.find_all('script', {'type':'application/ld+json'}):
        data = json.loads(i.get_text())
        if data['@type'] == 'ImageObject':
            url = data['mainEntityOfPage'].split('/car-exterior-image-1')[0]
            with open('all_models_url.jsonl', 'a') as f:
                json.dump(url, f)
                f.write('\n')

In [62]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
from tqdm import tqdm

max_workers = 10
urls = []
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(crawl, x) for x in brands]
    
    for future in tqdm(futures, total=len(brands)):
        result = future.result()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:05<00:00,  9.89it/s]
